__Defining Agent Types__

Daisy has three values aside from id and pos: age, breed (`:black` or `:white`), and albedo set by user. 
Land represents the surface, with the additional value of Temperature. Since the entire surface has the same albedo field, we don't need to set it as a Land value. 

*Note: Land does not necessarily have to be an agent, and you can represent surface temperature as a matrix (model parameter). This gives a performance advantage,*
*but incorporating surface agents allows for easier visualization*

In [2]:
using Agents, Plots 
using Statistics: mean

mutable struct Daisy <: AbstractAgent
    id::Int
    pos::Dims{2}
    breed::Symbol
    age::Int
    albedo::Float64 #0-1 Fraction
end

mutable struct Land <: AbstractAgent
    id::Int
    pos::Dims{2}
    temperature::Float64
end

const DaisyWorld = ABM{<:GridSpace, Union{Daisy, Land}};

In [7]:
function update_surface_temperature(pos::Dims{2}, model::DaisyWorld)
    ids = ids_in_position(pos, model)
    #All grid positions have at least one agent (land) = Set luminosity via surface albedo
    #More than one agent means daisy exists = Set luminosity via daisy albedo
    absorbed_luminosity = 
    (length(ids) == 1) ? (1-model.surface_albedo)*model.solar_liminosity : (1-model[ids[2]].albedo) * model.solar_liminosity
    #We expect heating to be 80 °C for an absorbed luminosity of 1,
    #approximately 30 for 0.5 and approximately -273 for 0.01.
    local_heating = (absorbed_luminosity > 0) ? 72 * log(absorbed_luminosity) + 80 : 80
    #Surface temperature is the average of the current temperature and local heating
    T0 = model[ids[1]].temperature 
    model[ids[1]].temperature = (T0 + local_heating) / 2
end


update_surface_temperature (generic function with 1 method)

In [10]:
function diffuse_temperature!(pos::Dims{2}, model::DaisyWorld)

ErrorException: syntax: invalid identifier name "?"